<a href="https://colab.research.google.com/github/busrakurunceng/phi4-finetuning/blob/main/phi_4_mini_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📄 Fine-tuning Phi-4 Mini Instruct

 Fine-tuning Phi-4 Mini Instruct Model

 This notebook demonstrates how to fine-tune the `microsoft/phi-4-mini-instruct` model using the Hugging Face `transformers` library.

 We will:
 - Load the model and tokenizer
 - Prepare a small dataset
 - Fine-tune the model
 - Save the fine-tuned model
 - Log the training loss in a clean tabular format

---

## 1. Install Required Libraries and Import Libraries

In [ ]:
!pip install transformers datasets accelerate peft trl bitsandbytes

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import TrainerCallback
from tabulate import tabulate

Download the model and tokenizer from Hugging Face.



In [ ]:

model_id = "microsoft/Phi-4-mini-instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_4bit=True,  # 4-bit modunda yükle
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    trust_remote_code=True
)


## 2.Install Model and Tokenizer

In [ ]:

# Modeli k-bit eğitime hazırla (4-bit olduğundan şart)
model = prepare_model_for_kbit_training(model)

# LoRA ayarları
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["self_attn.qkv_proj", "self_attn.o_proj", "mlp.gate_up_proj", "mlp.down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)


model = get_peft_model(model, lora_config)

## 3. Make Dataset

In [ ]:
# Örnek kendi veri kümesi
sample_data = [
    {
        "messages": [
            {"role": "user", "content": "What is machine learning?"},
            {"role": "assistant", "content": "Machine learning is a field of AI focused on building systems that learn from data."}
        ]
    },
    {
        "messages": [
            {"role": "user", "content": "Explain natural language processing."},
            {"role": "assistant", "content": "Natural language processing involves teaching machines to understand human language."}
        ]
    }
]

import json

# JSONL dosyasına kaydet
with open('train.jsonl', 'w') as f:
    for item in sample_data:
        f.write(json.dumps(item) + "\n")

# Dataseti yükle
dataset = load_dataset("json", data_files="train.jsonl")["train"]


## 4. Tokenize Dataset

In [ ]:
def tokenize_function(example):
    user_message = example['messages'][0]['content']
    assistant_message = example['messages'][1]['content']
    prompt = f"<|user|>\n{user_message}\n<|assistant|>\n{assistant_message}"
    result = tokenizer(prompt, truncation=True, padding="max_length", max_length=512)
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_dataset = dataset.map(tokenize_function, batched=False)

## 5. Define Custom Logging Callback
This callback formats the loss output in a clean table every logging step.

In [ ]:
class LoggingCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            table = [
                ["Step", state.global_step],
                ["Training Loss", round(logs.get("loss", 0.0), 4)]
            ]
            print(tabulate(table, headers=["Metric", "Value"], tablefmt="fancy_grid"))

## 6. Define Training Arguments

Set up training parameters: batch size, epochs, logging, and precision settings. Prepare the Hugging Face Trainer with the model, training arguments, and the dataset.


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./phi4-finetuned",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    num_train_epochs=2,
    logging_steps=20,
    save_steps=100,
    save_total_limit=1,
    fp16=True,
    report_to="none",
    disable_tqdm=True  # TQDM progress bar kapalı çünkü tablo basacağız
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    callbacks=[LoggingCallback()]
)



## 7. Start Training
Fine-tune the model on the prepared dataset!



In [ ]:
trainer.train()

## 8. Save the Fine-tuned Model
Save the fine-tuned model and tokenizer for future use.

In [ ]:
trainer.save_model("./phi4-finetuned-final")
tokenizer.save_pretrained("./phi4-finetuned-final")
